In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
from sklearn.datasets import make_friedman2
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import Kan_NN
import importlib
importlib.reload(Kan_NN)
import pickle
with open('fried_2_scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

In [8]:
from sklearn.preprocessing import StandardScaler
def get_loader(scaler_X, noise, n_samples = 20000):
    # Set the seed for reproducibility
    # Generate the Friedmann dataset
    #X_train, y = make_friedman1(n_samples= n_samples, n_features=in_dim, noise=noise)
    X_train, y = make_friedman2(n_samples= n_samples, noise=noise)
    y_train = np.expand_dims(y, axis=1)
    X_train = scaler_X.transform(X_train)  

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

    # Create TensorDataset for train and test sets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Set batch size and create DataLoader for training and testing
    batch_size = n_samples
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    return train_loader



In [9]:
@torch.no_grad()
def compute_test_loss(test_loader, model):
    criterion = torch.nn.MSELoss()
    running_loss = 0.
    i = 0 
    for batch, target in test_loader:
        i += 1 
        print(i, "/", len(test_loader))
        outputs = model(batch)
        loss = criterion(target, outputs)
        running_loss += loss.item()
    return running_loss / len(test_loader)

In [10]:
import warnings
warnings.simplefilter("ignore")

In [12]:
#### Evaluate models via for loop
import time
import dill
widths = [10] ## Grids
n_samples = 20000
in_dims = [4]
noises = [0.,0.2,0.5,1.]
train_losses = []
test_losses = []
for width in widths:
    for in_dim in in_dims:
        for noise in noises:
            test_loader = get_loader(scaler_X, noise = 0,n_samples =n_samples)
            with open(f"models/Friedmann_1_KAN_spline_arb_width_{noise}_{in_dim}.dill", "rb") as f:
                model = dill.load(f)
            loss = compute_test_loss(test_loader, model)
            print(width, noise, in_dim, loss)

1 / 1
10 0.0 4 0.05567643418908119
1 / 1
10 0.2 4 0.06539797782897949
1 / 1
10 0.5 4 0.08282063901424408
1 / 1
10 1.0 4 0.2480413019657135
